In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms.v2 as transforms
import os
import json
import logging
import numpy as np
from hyperparams import hypers
from models.yolo import Yolov1, Yolov2Tiny
from models.loss import YoloLossMultiBoxes
from train import train_fn
from dataprocess.dataset import *
from dataprocess.dataprocess import *
from dataprocess.dataprocess_coco import *
from test import *
from utils import *

import yaml
from ultralytics import YOLO

import logging
# logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")
logging.basicConfig(level=logging.INFO, format="%(levelname)s - %(message)s")

In [2]:

if hypers.DATASET_PATH == "LOCAL":
    drive_dir=os.path.join(os.getcwd(), "archive","data")
elif hypers.DATASET_PATH == "DRIVE":
    drive_dir=os.path.join(os.getcwd(),"drive","MyDrive","archive","data")



#Dataframe Creation
train_labels=os.path.join(drive_dir,"train_solution_bounding_boxes (1).csv")
train_imgs_dir=os.path.join(drive_dir,"training_images")
test_imgs_dir=os.path.join(drive_dir,"testing_images")

labels_dir=os.path.join(drive_dir,"label")
if os.path.exists(labels_dir) == False:
  os.mkdir(labels_dir)

images_dir=os.path.join(drive_dir,"image")
if os.path.exists(images_dir) == False:
  os.mkdir(images_dir)

#YOLOV8 PART START

yolo_dir=os.path.join(drive_dir,"yolov8")
if os.path.exists(yolo_dir) == False:
  os.mkdir(yolo_dir)

labels_dir_v8=os.path.join(yolo_dir,"labels")
if os.path.exists(labels_dir_v8) == False:
  os.mkdir(labels_dir_v8)

if os.path.exists(labels_dir_v8+"/train") == False:
  os.mkdir(labels_dir_v8+"/train")
if os.path.exists(labels_dir_v8+"/val") == False:
  os.mkdir(labels_dir_v8+"/val2")

images_dir_v8=os.path.join(yolo_dir,"images")
if os.path.exists(images_dir_v8) == False:
  os.mkdir(images_dir_v8)

if os.path.exists(images_dir_v8+"/train") == False:
  os.mkdir(images_dir_v8+"/train")
if os.path.exists(images_dir_v8+"/val") == False:
  os.mkdir(images_dir_v8+"/val")

prediction_dir=os.path.join(yolo_dir,"predictions")
if os.path.exists(prediction_dir) == False:
  os.mkdir(prediction_dir)

#COCO PART START
coco_dir=os.path.join(drive_dir,"coco")
if os.path.exists(coco_dir) == False:
  os.mkdir(coco_dir)

coco_dir_test=os.path.join(drive_dir,"coco_test")
if os.path.exists(coco_dir_test) == False:
  os.mkdir(coco_dir_test)

coco_weights=os.path.join(drive_dir,"coco_weights")
if os.path.exists(coco_weights) == False:
  os.mkdir(coco_weights)

#KAGGLE DATASET PART
imgs_list=list(sorted(os.listdir(train_imgs_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]


imgs_list=list(sorted(os.listdir(train_imgs_dir)))

#COCO DATASET PART

f = open('./annotations/instances_train2017.json')
data_coco = json.load(f)
f.close()
data_coco.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [3]:
#COCO TEST 
car_images=get_single_category(data_coco, 3)
car_images = sorted(list(set(car_images)))
# image_id=car_images[50]
# image_id_str_=str(image_id)
# image_id_pad=image_id_str_.zfill(12)
# # print(get_bbox(data_coco, image_id))
# url='http://images.cocodataset.org/train2017/'+image_id_pad+'.jpg'
# image = Image.open(requests.get(url, stream=True).raw)
# image
print(len(car_images))

12251


In [4]:
download_coco_images(car_images, 1000, coco_dir)
download_coco_images(car_images[5000:], 100, coco_dir_test)

100%|██████████| 100/100 [00:00<00:00, 7585.05it/s]


In [4]:
if len(list(sorted(os.listdir(labels_dir)))) == 0:
    logging.info("Populating v1 directories")
    populate_v1_dir(drive_dir, imgs_list, images_dir, labels_dir, train_imgs_dir)
    logging.info("Populated v1 directories")
    
if len(list(sorted(os.listdir(labels_dir_v8+"/train")))) == 0:
    logging.info("Populating v8 directories")
    populate_v8_dir(drive_dir, imgs_list, images_dir_v8, labels_dir_v8, train_imgs_dir, val_idx)
    logging.info("Populated v1 directories")
    
yaml_path = os.path.join(yolo_dir,"yolo.yaml")
if os.path.exists(yaml_path) == False:
    generate_yolov8_yaml(yolo_dir, images_dir_v8)

In [6]:
# if hypers.LOAD_MODEL_v8:
#     model = YOLO(hypers.path_best_weights_v8)
# else:
#     model=YOLO('yolov8m.pt')
# model.train(data=yolo_dir+'/yolo.yaml',epochs=20,patience=5,batch=8,
#                     lr0=0.0005,imgsz=640)

In [7]:
# # metrics = model.val()
# results, test_img_list = predict_yolo_v8(model, test_imgs_dir, prediction_dir)

# imgs_name=['vid_5_400','vid_5_26720']
# save_sample_pics(test_imgs_dir, imgs_name)

In [5]:
model = Yolov2Tiny(split_size=7, num_boxes=hypers.NUM_BOXES, num_classes=hypers.NUM_CLASSES).to(hypers.DEVICE)
optimizer = optim.Adam( model.parameters(), lr=hypers.LEARNING_RATE, weight_decay=hypers.WEIGHT_DECAY )
loss_fn = YoloLossMultiBoxes(S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES)

In [6]:
if hypers.DATA_AUGMENTATION:
    transform = Compose([
      transforms.RandomResizedCrop(size=(448, 448),scale=(hypers.MINIMUM_SCALE,1),ratio=(1,1), antialias=True),
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.ToTensor(),
      ])
else:
    transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

annotations=create_annotations(images_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt
annotations_predict=create_annotations(test_imgs_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt


dataset=VOCDataset(annotations, labels_dir, images_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)
dataset_predict=VOCDatasetPredict(annotations_predict, test_imgs_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)

dataset_coco = COCODataset(data=data_coco, transform=transform,image_folder=coco_dir,filter_category=[2], S=7, B=2, C=1)
dataset_coco_test = COCODataset(data=data_coco, transform=transform,image_folder=coco_dir_test,filter_category=[2], S=7, B=2, C=1)

train_set, test_set = torch.utils.data.random_split(dataset, [300, 55])

train_loader = DataLoader(dataset=train_set, batch_size=hypers.BATCH_SIZE,
                            pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

test_loader = DataLoader(dataset=test_set, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)


predict_loader = DataLoader(dataset=dataset_predict, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

coco_train_loader = DataLoader(dataset=dataset_coco, batch_size=hypers.BATCH_SIZE,
                            pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

coco_test_loader = DataLoader(dataset=dataset_coco_test, batch_size=hypers.BATCH_SIZE,
                            pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

C:\Users\ciarp\anaconda3\lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [ ]:
#YOLO TRAINING

map_array_train = []
map_array_test = []
for epoch in tqdm(range(10), position=0, leave=True):
    
    pred_boxes, target_boxes = get_bboxes(train_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
    mean_avg_prec_train = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
    pred_boxes, target_boxes = get_bboxes(test_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
    mean_avg_prec_test = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
    map_array_train.append(mean_avg_prec_train.item())
    map_array_test.append(mean_avg_prec_test.item())
    
    print(f"epoch: {epoch} Train mAP: {mean_avg_prec_train}")
    print(f"epoch: {epoch} Test mAP: {mean_avg_prec_test}")
    
    # k=0
    # if (mean_avg_prec > 0.9) and (k==0):
    #     k=1
    #     checkpoint = {
    #         "state_dict": model.state_dict(),
    #         "optimizer": optimizer.state_dict(),
    #     }
    #     save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE, exit_training=True)
    #     # time.sleep(10)

    train_fn(train_loader, model, optimizer, loss_fn)
    


In [ ]:
#COCO TRAINING
# map_array_train = []
# map_array_test = []
# for epoch in tqdm(range(10), position=0, leave=True):
    
#     if epoch % 2 == 0:
        
#         pred_boxes, target_boxes = get_bboxes(coco_train_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
#         mean_avg_prec_train = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
#         # pred_boxes, target_boxes = get_bboxes(coco_test_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
#         # mean_avg_prec_test = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")
    
#         map_array_train.append(mean_avg_prec_train.item())
#         # map_array_test.append(mean_avg_prec_test.item())
    
#         print(f"epoch: {epoch} Train mAP: {mean_avg_prec_train}")
#         # print(f"epoch: {epoch} Test mAP: {mean_avg_prec_test}")
        
#         if mean_avg_prec_train >= max(map_array_train):
#             checkpoint = {
#                 "state_dict": model.state_dict(),
#                 "optimizer": optimizer.state_dict(),
#             }
#             save_checkpoint(checkpoint, filename=coco_weights+"/best_train_weights.pth", exit_training=False)
            
#         # if mean_avg_prec_test >= max(map_array_test):
#         #     checkpoint = {
#         #         "state_dict": model.state_dict(),
#         #         "optimizer": optimizer.state_dict(),
#         #     }
#         #     save_checkpoint(checkpoint, filename=coco_weights+"/best_test_weights.pth", exit_training=False)
        
        
    

#     train_fn(coco_train_loader, model, optimizer, loss_fn)
# checkpoint = {
#                 "state_dict": model.state_dict(),
#                 "optimizer": optimizer.state_dict(),
#             }
# save_checkpoint(checkpoint, filename=coco_weights+"/last_train_weights.pth", exit_training=False)


In [6]:
load_checkpoint(torch.load(coco_weights+"/best_train_weights.pth"), model, optimizer)

=> Loading checkpoint


In [13]:
pred_boxes, target_boxes = get_bboxes(coco_train_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
mean_avg_prec_train = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")

# pred_boxes, target_boxes = get_bboxes(coco_test_loader, model, iou_threshold=0.5, threshold=0.4,S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, device=hypers.DEVICE)
# mean_avg_prec_test = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint")

# map_array_train.append(mean_avg_prec_train.item())
# map_array_test.append(mean_avg_prec_test.item())

# print(f"epoch: {epoch} Train mAP: {mean_avg_prec_train}")
# print(f"epoch: {epoch} Test mAP: {mean_avg_prec_test}")

In [14]:
print(f"Train mAP: {mean_avg_prec_train}")


Train mAP: 0.2536362111568451


In [ ]:
plot_real_images(data_coco, coco_dir_test, end_index=20)

In [ ]:
result=model.predict(coco_dir,conf=0.50,iou=0.2)
plot_predicted_images(coco_dir, result)


In [ ]:
result=model.predict(coco_dir_test,conf=0.50,iou=0.2)
plot_predicted_images(coco_dir_test, result)

In [ ]:
result=model.predict(coco_dir,conf=0.50,iou=0.2)
plot_predicted_images(coco_dir, result)